In [4]:
def discreteLogarithm(p, a, b, G, B, A):
    '''
        Discrete Logarithm for a curve that its parameter is smooth. 
        p: prime number 
        a, b: parameters of the curve
        G: generator point
        B: Bob's public key
        A: Alice's public key
    '''
    E = EllipticCurve(GF(p), [a, b])
    G = E(G)
    B = E(B)
    A = E(A)
    b = G.discrete_log(B)
    sharedSecret = (A*b).xy()[0]
    return sharedSecret

def test():
    p = 310717010502520989590157367261876774703
    a = 2 
    b = 3
    G = [179210853392303317793440285562762725654, 105268671499942631758568591033409611165]
    B = [272640099140026426377756188075937988094, 51062462309521034358726608268084433317]
    A = [280810182131414898730378982766101210916, 291506490768054478159835604632710368904]
    print(discreteLogarithm(p, a, b, G, B, A))

In [6]:
def _lift(E, P, gf):
    # Using Hansel's point lifting.
    x, y = map(ZZ, P.xy())
    for point_ in E.lift_x(x, all=True):
        _, y_ = map(gf, point_.xy())
        if y == y_:
            return point_ 

def attack(G, P):
    '''
        Solves discrete logarithm by Smart's Attack.
        G: base point
        P: a point that have such nP where nP*G == P
        return: nP
    '''
    E = G.curve()
    gf = E.base_ring()
    p = gf.order()
    assert E.trace_of_frobenius() == 1, f"Curve should have trace of Frobenius = 1."

    E = EllipticCurve(Qp(p), [int(a) + p * ZZ.random_element(1, p) for a in E.a_invariants()])
    G = p * _lift(E, G, gf)
    P = p * _lift(E, P, gf)
    Gx, Gy = G.xy()
    Px, Py = P.xy()
    return int(gf((Px / Py) / (Gx / Gy)))

In [8]:
# Small subgroup attack in ECDH
# Given a curve E(F_p), we will transfer it to an additive group of F_p (aka finding group isomorphism)
# Method using: Smart's Attack


def test():
    '''
        Define the elliptic curve, public keys of both 2 parties 
    '''
    p = 0xa15c4fb663a578d8b2496d3151a946119ee42695e18e13e90600192b1d0abdbb6f787f90c8d102ff88e284dd4526f5f6b6c980bf88f1d0490714b67e8a2a2b77
    a = 0x5e009506fcc7eff573bc960d88638fe25e76a9b6c7caeea072a27dcd1fa46abb15b7b6210cf90caba982893ee2779669bac06e267013486b22ff3e24abae2d42
    b = 0x2ce7d1ca4493b0977f088f6d30d9241f8048fdea112cc385b793bce953998caae680864a7d3aa437ea3ffd1441ca3fb352b0b710bb3f053e980e503be9a7fece
    E = EllipticCurve(GF(p), [a, b])
    G = E(3034712809375537908102988750113382444008758539448972750581525810900634243392172703684905257490982543775233630011707375189041302436945106395617312498769005,4986645098582616415690074082237817624424333339074969364527548107042876175480894132576399611027847402879885574130125050842710052291870268101817275410204850)
    A = E(4748198372895404866752111766626421927481971519483471383813044005699388317650395315193922226704604937454742608233124831870493636003725200307683939875286865, 2421873309002279841021791369884483308051497215798017509805302041102468310636822060707350789776065212606890489706597369526562336256272258544226688832663757)
    nA = attack(G, A) # Finding Alice private key 
    assert nA*G == A
    print(nA)
test()

2200911270816846838022388357422161552282496835763864725672800875786994850585872907705630132325051034876291845289429009837283760741160188885749171857285407
